Para entrar no modo apresentação, execute a seguinte célula e pressione `-`

In [ ]:
%reload_ext slide

<span class="notebook-slide-start"/>

# Git

Este notebook apresenta os seguintes tópicos:

- [Git](#Git)
- [Exercício 9](#Exerc%C3%ADcio-9)
- [Pandas](#Pandas)
- [Exercício 10](#Exerc%C3%ADcio-10)

## Git

Outra fonte de informações de um repositório de software é o repositório do sistema de controle de versões.

Pelo controle de versões, conseguimos ter acesso a todos os arquivos de todas as versões, todas as mensagens de commit, branches, e colaboradores.

Nesta parte do minicurso, faremos a mineração dessas informações.

No caso do Git, ao clonar um repositório, ficamos com uma cópia local do que está lá. Portanto, começamos a mineração com um clone e não precisamos de nenhum proxy.  <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
!git clone https://github.com/gems-uff/sapos

Com o repositório clonado, podemos usar comandos do git para extrair informações.  <span class="notebook-slide-extra" data-count="2"/>

In [ ]:
%cd sapos

In [ ]:
!git branch -r

Essas informações também podem ser obtidas para tratarmos usando variáveis do Python.

A seguir tentamos descobrir qual é o commit de cada um desses branches. <span class="notebook-slide-extra" data-count="2"/>

In [ ]:
git_branch_output = !git branch -r
branches = [
    branch.strip().split(' ')[0].split('/')[1]
    for branch in git_branch_output
]
branches

In [ ]:
branch_commit = {}
for branch in branches:
    __ = !git checkout $branch
    commit = !git show --pretty=format:"%h" --no-patch
    branch_commit[branch] = commit
__ = !git checkout master
branch_commit

Usamos `__ = !...` para evitar a exibição do output do comando de sistema. O IPython imprime o output quando bang expressions são usadas isoladas e retorna o output quando elas são usadas em atribuições.

Note que apenas o branch `reports` está em um commit diferente.  <span class="notebook-slide-scroll" data-position="-1"/>

## Exercício 9

Faça a mesma operação para obter o código dos commits de tags e salve na variável `tag_commit`. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
tags = !git tag
...

Agora vamos agrupar as tags por versões minor e ordenar as versões patch. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
from itertools import groupby
groups = groupby(tags, lambda x: x.rsplit(".", 1)[0])
minor_tags = {}
for minor, elements in groups:
    minor_tags[minor] = sorted(
        elements,
        key=lambda x: int(x.split('-')[0].split('.')[-1])
    )
minor_tags['4.3']

Fazendo o mesmo para agrupar versões major. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
groups = groupby(minor_tags, lambda x: x.rsplit(".", 1)[0])
major_tags = {}
for major, elements in groups:
    major_tags[major] = sorted(
        elements,
        key=lambda x: int(x.split('-')[0].split('.')[-1])
    )
major_tags['4']

Com isso, podemos escolher versões major (e.g., 3 e 4) e obter a última versão patch para cada minor delas. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
last_patch_for_v3v4 = {
    minor: minor_tags[minor][-1]
    for minor in major_tags['3'] + major_tags['4']
}
last_patch_for_v3v4

Agora queremos ver a evolução de linhas de código para as versões selecionadas. Para isso, vamos percorrer o dicionário fazendo checkout de cada versão, carregar o número de linhas usando `cloc` e parsear o resultado para extrair as colunas para construir linhas de uma tabela. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
from collections import defaultdict
columns = {"id"}
rows = []

for minor, tag in last_patch_for_v3v4.items():
    __ = !git checkout $tag
    lines = !cloc .
    filtered_lines = lines[lines.index("-" * 79) + 3:]
    commit_result = defaultdict(int)
    commit_result["id"] = minor
    for line in filtered_lines:
        if not line.startswith("-"):
            split = line.split()
            language = split[0]
            commit_result[language + "_files"] = int(split[1])
            commit_result[language + "_blank"] = int(split[2])
            commit_result[language + "_comment"] = int(split[3])
            commit_result[language + "_code"] = int(split[4])
            columns |= {
                language + "_files", language + "_blank",
                language + "_comment", language + "_code"
            }
    rows.append(commit_result)

## Pandas
Podemos usar `pandas` para construir a tabela a partir da lista de dicionários.  <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
import pandas as pd
df = pd.DataFrame(rows)
df

O `pandas` permite descrever a tabela com o método `.describe()`.  <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
df.describe()

Além disso, é possível fazer seleções nos dados. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
df[df["Ruby_code"] > 25000]

## Exercício 10

Selecione as versões que usam CoffeeScript e as versões que não usam XML.


Além de selecionar linhas, podemos selecionar colunas. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
columns = ['SUM:_files', 'SUM:_blank', 'SUM:_comment', 'SUM:_code']
ndf = df[columns]
ndf

O `pandas` também oferece algumas funções que facilitam a geração de gráficos. <span class="notebook-slide-extra" data-count="2"/>

In [ ]:
%matplotlib inline
ndf.boxplot()

In [ ]:
df.set_index("id")["Ruby_code"].plot()

É possível aplicar operações em colunas e criar novas colunas. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
df.loc[:, "tag"] = df["id"].apply(lambda minor: last_patch_for_v3v4[minor])

In [ ]:
df

Existem muitas outras operações que podem ser vistas na documentação: https://pandas.pydata.org/pandas-docs/stable/. <span class="notebook-slide-scroll" data-position="-1"/>

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;


&nbsp;

&nbsp;

&nbsp;

